# 01. EDA

### Importando as bibliotecas

In [1]:
import sys
import os
sys.path.append('..')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras

### Configurando para serem exibidas apenas mensagens de erro no Tensor Flow

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Configurando para serem exibidas apenas mensagens de erro no Tensor Flow

### Lendo o dataset e visualizando um overview

In [3]:
df = keras.datasets.imdb # Armazenando o dataset em uma variável

In [4]:
(x_train, y_train), (x_test, y_test) = df.load_data() # Armazenando os dados do df já divididos em X e Y de treino e teste

In [5]:
x_train.shape, y_train.shape # Verificando a quantidade de avaliações nos conjuntos X e Y de treino

((25000,), (25000,))

In [6]:
x_test.shape, y_test.shape # Verificando a quantidade de avaliações nos conjuntos X e Y de teste

((25000,), (25000,))

### Verificando o balanceamento do dataset

In [7]:
np.unique(y_train) # Verificando se realmente existem os número de 0 a 9

array([0, 1], dtype=int64)

In [8]:
np.unique(y_train, return_counts=True) # Verificando a distribuição de valores

(array([0, 1], dtype=int64), array([12500, 12500], dtype=int64))

In [9]:
contagem = tuple(zip(*np.unique(y_train, return_counts=True))) # Criando tuplas com o unpacking dos arrays, através do zip

for tupla in contagem: # Criando uma estrutura de repetição para calcular a porcentagem de cada número no conjunto y de treino
    print(f'{tupla[0]}: {tupla[1] / len(y_train) * 100:.2f}%')

0: 50.00%
1: 50.00%


Após a análise, é verificado que o dataset está balanceado e não é necessário aplicar nenhum tratamento para datasets desbalanceados.

### Entendendo o dataset

In [10]:
for review in x_train[:5]: # Criando uma estrutura de repetição para printar o número de tokens dos 5 primeiros reviews
    print((len(review))) # Printando o número de tokens

218
189
141
550
147


In [11]:
x_train[0] # Visualizando como o review está, sendo vários valores que possuem uma chave correspondente para serem decodificados por um dicionário de palavras

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 22665,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 21631,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 19193,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 5244,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 10311,
 8,
 4,
 107,
 117,
 5952,
 15,
 256,
 4,
 31050,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 12118,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 7486,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5

In [12]:
word_index = df.get_word_index() # Armazenando o dicionário de palavras em uma variável

In [13]:
len(word_index) # Verificando a quantidade de palavras do dicionário

88584

In [14]:
word_index = {chave: (valor + 3) for chave, valor in word_index.items()} # Adicionando 4 espaços no dicionário para inserir tokens especiais para NLP: PAD, START, UNK e UNUSED

Tokens especiais:
- PAD é o token usado para preenchimento. Nós preenchemos todas as sequências para o mesmo comprimento, que é o comprimento da sequência mais longa.
- START é o token usado para marcar o início de uma sequência.
- UNK é o token usado para marcar palavras desconhecidas (palavras que não estão no vocabulário).
- UNUSED é o token usado para preencher posições não utilizadas em uma sequência.

In [15]:
# Adicionando os tokens especiais em suas respectivas posições
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2
word_index['<UNUSED>'] = 3

In [16]:
def decode_review(text, index): # Criando uma função para transformar os reviews, de sequência de números para um texto normal que pode ser lido, passando um texto e o dicionário de palavras
    reverse_index = {value: key for key, value in index.items()} # Invertendo o dicionário anterior de palavras para números, para números para palavras
    return ' '.join([reverse_index.get(value, '<UNK>') for value in text]) # Retorna o texto do review, percorrendo cada número e buscando sua palavra correspondente

In [17]:
decoded_text = decode_review(x_train[0], word_index) # Usando a função para decodificar e visualizar o primeiro review, passando um review e o dicionário de palavras
print('\n'.join([decoded_text[i:i+100] for i in range(0, len(decoded_text), 100)])) # Printando o texto com limite de 100 caracteres

<START> this film was just brilliant casting location scenery story direction everyone's really suit
ed the part they played and you could just imagine being there robert redford's is an amazing actor 
and now the same being director norman's father came from the same scottish island as myself so i lo
ved the fact there was a real connection with this film the witty remarks throughout the film were g
reat it was just brilliant so much that i bought the film as soon as it was released for retail and 
would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it w
as so sad and you know what they say if you cry at a film it must have been good and this definitely
 was also congratulations to the two little boy's that played the part's of norman and paul they wer
e just brilliant children are often left out of the praising list i think because the stars that pla
y them all grown up are such a big profile for the whole film but these children are amazin

In [18]:
x_train = keras.utils.pad_sequences( # Ajustando o novo valor de x de treino para otimizar os recursos de processamento, limitando os reviews em 256 tokens (nesse caso, palavras). Para reviews menores, serão preenchidos com <PAD>. Para reviews maiores, serão cortados em 256 tokens.
    x_train, # Passando os dados que serão alterados
    value=word_index['<PAD>'], # Passando o valor que preenchera os espaços vazios caso os reviews tenham menos de 200 tokens
    padding='post', # Definindo se os reviews com menos de 256 tokens, vão ser preenchidos com <PAD> no início ou no final
    truncating='post', # Definindo se os reviews com mais de 256 tokens, vão ser cortados no início ou no final
    maxlen=256 # Definindo o número máximo de tokens
)

In [19]:
x_test = keras.utils.pad_sequences( # Ajustando o novo valor de x de teste para otimizar os recursos de processamento, limitando os reviews em 256 tokens (nesse caso, palavras). Para reviews menores, serão preenchidos com <PAD>. Para reviews maiores, serão cortados em 256 tokens.
    x_test, # Passando os dados que serão alterados
    value=word_index['<PAD>'], # Passando o valor que preenchera os espaços vazios caso os reviews tenham menos de 256 tokens
    padding='post', # Definindo se os reviews com menos de 256 tokens, vão ser preenchidos com <PAD> no início ou no final
    truncating='post', # Definindo se os reviews com mais de 256 tokens, vão ser cortados no início ou no final
    maxlen=256 # Definindo o número máximo de tokens
)

In [24]:
decoded_text = decode_review(x_train[1], word_index) # Verificando se a definição no número de tokens em 200 funcionou corretamente
print('\n'.join([decoded_text[i:i+100] for i in range(0, len(decoded_text), 100)])) # Printando o texto com limite de 100 caracteres

<START> big hair big boobs bad music and a giant safety pin these are the words to best describe thi
s terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the
 worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is co
mpletely laughable the best is the end showdown with the cop and how he worked out who the killer is
 it's just so damn terribly written the clothes are sickening and funny in equal measures the hair i
s big lots of boobs bounce men wear those cut tee shirts that show off their stomachs sickening that
 men actually wore them and the music is just synthesiser trash that plays over and over again in al
most every scene there is trashy music boobs and paramedics taking away bodies and the gym still doe
sn't close for bereavement all joking aside this is a truly bad film whose only charm is to look bac
k on the disaster that was the 80's and have a good old laugh at how bad everything was bac

In [21]:
for review in x_train[:5]: # Criando uma estrutura de repetição para printar o número de tokens dos 5 primeiros reviews
    print((len(review))) # Printando o número de tokens

256
256
256
256
256


In [22]:
x_train.shape # Verificando a quantidade de avaliações nos conjuntos X de treino e a quantidade de tokens em cada review

(25000, 256)